In [3]:
import asyncio

shell = get_ipython()
kernel = shell.kernel
kernel

In [ ]:
from ipykernel.eventloops import register_integration
EVENT = asyncio.Event()
CAPTURED_MESSAGES = []

def get_messages(stream):
    poller = zmq.Poller()
    poller.register(stream.socket, zmq.POLLIN)
    while True:
        ident, msg = kernel.session.recv(stream, zmq.NOBLOCK, content=True)
        if msg is None:
            return
        captured_messages.append((stream, ident, msg))

@register_integration('block')
def loop_block(kernel):
    "Capture all queued up ZMQ Poll events, then block until global asyncio EVENT is set()"
    global EVENT
    global CAPTURED_MESSAGES
    EVENT = asyncio.Event() # we can use this to block in a run_until_complete(EVENT.wait())
    CAPTURED_MESSAGES = [] # all other 'cells' that have been run
    loop = asyncio.get_event_loop()
    # loop is already running (e.g. tornado 5), nothing left to do
    if loop.is_running():
        return

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    while True:
        error = None
        try:
            loop.run_forever()
        except KeyboardInterrupt:
            continue
        except Exception as e:
            error = e
        if hasattr(loop, 'shutdown_asyncgens'):
            loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()
        if error is not None:
            raise error
        break

In [21]:
kernel.control_stream

In [4]:
loop = asyncio.get_event_loop()
kernel.eventloop = loop
kernel

In [5]:
kernel.eventloop

<_WindowsSelectorEventLoop running=False closed=False debug=False>

In [6]:
kernel.do_one_iteration()

In [15]:
import ipywidgets as widgets

w = widgets.Dropdown(options=['', 'foo', 'bar', 'baz'])
w.observe(lambda change: print(change), 'value')
w

Dropdown(options=('', 'foo', 'bar', 'baz'), value='')

{'name': 'value', 'old': '', 'new': 'foo', 'owner': Dropdown(index=1, options=('', 'foo', 'bar', 'baz'), value='foo'), 'type': 'change'}
{'name': 'value', 'old': 'foo', 'new': 'bar', 'owner': Dropdown(index=2, options=('', 'foo', 'bar', 'baz'), value='bar'), 'type': 'change'}


In [18]:
import time

for i in range(10):
    kernel.do_one_iteration()
    time.sleep(2)

In [19]:
print('hello world')

hello world


In [20]:
print('test')

test
